In [6]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
import os
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import scorer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn import preprocessing

#some deprecation notices
import warnings
warnings.filterwarnings("ignore")

#set a seed so results are replicable 
import random
random.seed(121)

In [7]:
def prepare_data_from_df_cv(data, class_name):

    """
    For classifying with cross-validation
    data(df): Pandas dataframe with all data, with last column containing the classification labels and all 
                others as features. Row names are sampleIDs.    
    class_name(str): name of the classification column

    Returns: 
        X (df): the 16S data
        y (Series): the factored classes
        factor_index: the factorized index for each class
        features (list): list of 16S feature names
    """
    df = data.copy()
    features = df.columns[:-1]
    df["factored"], factor_index = pd.factorize(df[class_name])
    X, y = df[features], df["factored"]
    
    return X, y, factor_index, features

def prepare_data_from_df(data, class_name, train_split=0.75):
    """
    For classifying without cross-validation
    data(df): Pandas dataframe with all data, with last column containing the classification labels and all 
                others as features. Row names are sampleIDs.
    train_split(float): percent of data to use as training set (default 0.75)
    
    class_name(str): name of the classification column
    
    
    """
    df = data.copy()
    features = df.columns[:-1]
    df["factored"], factor_index = pd.factorize(df[class_name])
    
    df['is_train'] = np.random.uniform(0, 1, data.shape[0]) <= train_split
    train, test = df[df['is_train']==True], df[df['is_train']==False]
    Xtrain, Xtest, ytrain, ytest = train[features], test[features], train["factored"], test["factored"]
    
    return Xtrain, Xtest, ytrain, ytest, factor_index, features
    
def train_and_test_classifier(Xtrain, Xtest, ytrain, ytest, features, max_features, n_estimators=len(features)):
    """
    Xtrain(df): training data - row names are sampleIDs
    Xtest(df): test data - row names are sampleIDs
    ytrain(df): correct classification of training samples (factorized) - generally last column of a dataset
    ytest(df): correct classification of test samples (factorized) - generally last column of a dataset
    
    Returns:
        clf(RandomForestClassifier): the fitted classifier
        acc: the accuracy of the classifier
    """
    clf = RandomForestClassifier(n_jobs=-1, n_estimators=n_estimators, max_features=max_features)
    clf.fit(Xtrain[features], ytrain)
    acc = clf.score(Xtest[features], ytest) 
    
    return clf, acc

In [ ]:
def classifier_analysis(data, metadata, runs=100, cv=10, feature_runs=100, num_features=5, class_name="diabetes"):
    """
    Produces in the current directory 1) a text file of average AUC for each class comparison 
    (T2D vs NGT, IGT vs NGT, T2D vs IGT), 2) a table of the most important num_features features in each of feature_runs runs,
    for each class comparison, with the importance of each feature for that run.
    
    data (df): otu table (samples, features), with a descriptive data.index.name
    metadata (df): clinical data (samples, features)
    
    Returns: data.index.name
    """
    
    #prepare the data, making sure that there are the same samples in the 16S data and the metadata
    data.index = data.index.map(str)
    metadata = metadata[metadata.index.isin(data.index)]
    data = data[data.index.isin(metadata.index)]
    data_classifier = data.copy()
    #get the diabetes classification for each sample in the 16S data
    data_classifier["diabetes"] = metadata["diabetes"]
    
    #separate the different classes of samples
    NGT = data_classifier[data_classifier["diabetes"]=="NGT"]
    T2D = data_classifier[data_classifier["diabetes"]=="T2D"]
    IGT = data_classifier[data_classifier["diabetes"]=="IGT"]
    T2D.index.name = "T2D"
    NGT.index.name = "NGT"
    IGT.index.name = "IGT"    
    
    with open("{}_mean_auc_cv10.txt".format(data.index.name), "w") as auc_txt:
        for class_pair in [[T2D, NGT_T2D_sample], [T2D, IGT_T2D_sample], [IGT, NGT_IGT_sample]]:
            with open("{}_{}_{}_important_features_cv10.csv".format(data.index.name, class_pair[0].index.name, class_pair[1].index.name), "w") as features_txt:
                cv_auc = []
                data_sample = pd.concat(class_pair)
                X,y, factor_index, features = prepare_data_from_df_cv(data_sample, class_name)                
                #take the average AUC across the runs (with cross-validation)
                for i in range(runs):
                    clf = RandomForestClassifier(n_jobs=-1, n_estimators=len(features), max_features=len(features))                    
                    cv_auc.append(cross_val_score(clf, X, y, cv=cv,n_jobs=-1, scoring=make_scorer(roc_auc_score)))
                auc_txt.write("{} vs {}: {}".format(class_pair[0].index.name, class_pair[1].index.name, np.mean(cv_auc)))
                auc_txt.write("\n")
                important_features = pd.DataFrame()
                #getting the most important features for each run (no cross-validation), with a different split each time
                for i in range(feature_runs):
                    Xtrain, Xtest, ytrain, ytest, factor_index, features = prepare_data_from_df(data_sample, class_name)
                    clf, acc = train_and_test_classifier(Xtrain, Xtest, ytrain, ytest, features, len(features))
                    important_features = important_features.append(pd.DataFrame(index=features, columns=["importance"], data=clf.feature_importances_).sort_values(by="importance", ascending=False).head())
                important_features.to_csv("{}_{}_{}_important_features.csv".format(data.index.name, class_pair[0].index.name, class_pair[1].index.name))   
    return data.index.name

In [52]:
variants = pd.read_csv("../christien_tables/DESeq_normalized_counts/counts_norm_variant.csv", index_col=0)
variants.index.name = "variants"
genera = pd.read_csv("../christien_tables/DESeq_normalized_counts/counts_norm_genera.csv", index_col=0)
genera.index.name = "genera"
family = pd.read_csv("../abundances_by_tax_level/Family_otu_table.csv", index_col=0).T
family.index.name = "family"
sparcc = pd.read_csv("../sparcc/sparcc_clustered_otu.csv", index_col=0).T 
sparcc.index.name = "sparcc"
sparcc = sparcc/sparcc.sum()

In [53]:
data = [family, sparcc, genera, variants]

In [30]:
#classifying diabetic status based on WHO standards
metadata = pd.read_csv("../metadata_variable_filtered/metadata_for_pcoa_num_only_nosum.csv", index_col=0) #read metadata file
metadata.index = metadata.index.map(str)
metadata["diabetes"] = np.zeros
for index in metadata.index:
    fasting = metadata.loc[index, "glucose_0"]
    twohr = metadata.loc[index, "glucose_120"]
    if fasting >= 126 or twohr >= 200:
        metadata["diabetes"][index] = "T2D"
    else:
        if twohr >= 140 and twohr < 200:
            metadata["diabetes"][index] = "IGT"
        else:
            metadata["diabetes"][index] = "NGT"

In [ ]:
for df in data:
    print(classifier_analysis(df, metadata, runs=100, cv=10))

family
sparcc
genera
